In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
url='https://raw.githubusercontent.com/digipodium/Datasets/main/regression/50_Startups.csv'
df = pd.read_csv(url)
df

eda

In [ ]:
df.isnull().sum()

In [ ]:
import seaborn as sns

In [ ]:
sns.regplot(x='R&D Spend',y='Profit',data=df) # y is target variable

In [ ]:
sns.regplot(x='Administration',y='Profit',data=df)

In [ ]:
sns.regplot(x='Marketing Spend',y='Profit',data=df)

In [ ]:
sns.pairplot(df, kind='reg', diag_kind='kde') # this is not good for large dataset

In [ ]:
sns.boxplot(x='State',y='Profit',data=df)

In [ ]:
df.select_dtypes('number').corrwith(df['Profit']).plot(kind='bar')

training

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [ ]:
X = df.drop(columns=['Profit'])
y = df['Profit']

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, 
                                            test_size=0.2, 
                                            random_state=0)

In [ ]:
cat_col = X.select_dtypes('object').columns
num_col = X.select_dtypes('number').columns
cat_pipe = Pipeline(steps=[
    ('ohe', OneHotEncoder(drop='first'))
])
num_pipe = Pipeline(steps=[
    ('scaler', StandardScaler())
])
preprocessor = ColumnTransformer(transformers=[
    ('num', num_pipe, num_col),
    ('cat', cat_pipe, cat_col)
])
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('lr', LinearRegression())
])

In [ ]:
model.fit(Xtrain, ytrain)

In [ ]:
train_ypred = model.predict(Xtrain)
test_ypred = model.predict(Xtest)
print(f"train r2 score: {r2_score(ytrain, train_ypred)}")
print(f"test r2 score: {r2_score(ytest, test_ypred)}")
print(f"train mse: {mean_squared_error(ytrain, train_ypred)}")
print(f"test mse: {mean_squared_error(ytest, test_ypred)}")
print(f"train mae: {mean_absolute_error(ytrain, train_ypred)}")
print(f"test mae: {mean_absolute_error(ytest, test_ypred)}")

In [ ]:
sns.kdeplot(ytest, label='ytest')
sns.kdeplot(test_ypred, label='test_ypred')
plt.legend()
plt.show()

In [35]:
from joblib import dump, load

with open('models/50_startup_model.pkl', 'wb') as f:
    dump(model, f)
    print('✅ Model is saved')

✅ Model is saved


gradio app
`pip install gradio`

In [39]:
X.head(1)

,R&D Spend,Administration,Marketing Spend,State
0,165349.2,136897.8,471784.1,New York


In [41]:
import gradio as gr
import pandas as pd
def prediction(rnd, admin, marketing, state):
    # make a single record dataframe
    df = pd.DataFrame({
        'R&D Spend': [rnd],
        'Administration': [admin],
        'Marketing Spend': [marketing],
        'State': [state]
    })
    # load joblib model file
    with open('models/50_startup_model.pkl', 'rb') as f:
        model = load(f)
    # predict the profit
    profit = model.predict(df) # returns numpy array
    return profit[0] # return the profit value

ui = gr.Interface(
    fn = prediction,
    inputs = ['number', 'number', 'number', 'text'],
    outputs = gr.Text(),
    title = '50 Startup Profit Prediction',
    examples=[
        [165349, 136897, 471784, 'New York']
    ]
)
ui.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
